In [1]:
%load_ext autoreload
%autoreload 2
from abc import ABC, abstractmethod
from trade.helpers.Logging import setup_logger
from pydantic.dataclasses import dataclass as pydantic_dataclass
from pydantic import Field, ConfigDict
import pandas as pd
from pandas.tseries.offsets import BDay
from typing import ClassVar
from weakref import WeakSet
from datetime import datetime
import numbers
from typing import Optional, Dict, Any, Type, Tuple, List, Union, get_args, get_origin

logger = setup_logger(__name__, stream_log_level="DEBUG")

class BacktesterIncorrectTypeError(Exception):
    pass



Console Logging & File Logging Can be configured using STREAM_LOG_LEVEL and FILE_LOG_LEVEL in environment variables.
Propagate to root logger can be set using PROPAGATE_TO_ROOT_LOGGER in environment variables.
Example:
STREAM_LOG_LEVEL = 'DEBUG'
FILE_LOG_LEVEL = 'INFO'
PROPAGATE_TO_ROOT_LOGGER = 'False'

2025-11-16 17:59:50 trade.helpers.Logging INFO: Logging Root Directory: /Users/chiemelienwanisobi/cloned_repos/QuantTools/logs


In [8]:
from EventDriven.configs.base import BaseConfigs

In [14]:


@pydantic_dataclass(config=ConfigDict(arbitrary_types_allowed=True))
class ChainConfigs(BaseConfigs):
    """
    Configuration class for Option Chain related settings.
    Ultimately, it would be used to filter the chain data retrieved from the data source.
    """
    max_pct_width: numbers.Number = None
    min_oi: numbers.Number = None



chain_conf = ChainConfigs()
# chain_conf.display_and_describe_configs()
chain_conf.max_pct_width=1

2025-11-16 18:08:57 EventDriven.configs.base WARNING: Configuration 'min_oi' is not set (None). Please review.


In [12]:
@pydantic_dataclass(config=ConfigDict(arbitrary_types_allowed=True))
class OrderSchemaConfigs(BaseConfigs):
    """
    Configuration class for Order Schema related settings.
    """
    target_dte: numbers.Number = 270
    strategy: str = "vertical"
    structure_direction: str = "long"
    spread_ticks: numbers.Number = 1
    dte_tolerance: numbers.Number = 60
    min_moneyness: numbers.Number = 0.65
    max_moneyness: numbers.Number = 1
    min_total_price: numbers.Number = 0.95

order_schema_conf = OrderSchemaConfigs(dte_tolerance=120)
order_schema_conf.display_and_describe_configs()


Current Configuration Settings for OrderSchemaConfigs:
{'target_dte': 270, 'strategy': 'vertical', 'structure_direction': 'long', 'spread_ticks': 1, 'dte_tolerance': 120, 'min_moneyness': 0.65, 'max_moneyness': 1, 'min_total_price': 0.95}
        

Configuration Descriptions for OrderSchemaConfigs:
- target_dte: 270  # Target days to expiration for the options in the order schema.
- strategy: vertical  # The options strategy to be used (e.g., vertical, iron condor).
- structure_direction: long  # Direction of the structure, either long or short.
- spread_ticks: 1  # Number of strike price ticks between legs of the spread.
- dte_tolerance: 120  # Allowed deviation in days to expiration from the target DTE.
- min_moneyness: 0.65  # Minimum moneyness level for selecting options.
- max_moneyness: 1  # Maximum moneyness level for selecting options.
- min_total_price: 0.95  # Minimum total price for the option structure.



In [13]:
@pydantic_dataclass(config=ConfigDict(arbitrary_types_allowed=True))
class OrderPicker(BaseConfigs):
    """
    Configuration class for Order Picker related settings.
    """
    start_date: Union[str,datetime]
    end_date: Union[str,datetime]

    def __post_init__(self, ctx=None):
        super().__post_init__(ctx)
        # Convert start_date and end_date to datetime if they are strings
        if isinstance(self.start_date, str):
            self.start_date = pd.to_datetime(self.start_date).date()
        if isinstance(self.end_date, str):
            self.end_date = pd.to_datetime(self.end_date).date()

order_picker_conf = OrderPicker(
    start_date="2020-01-01",
    end_date="2024-01-01"
)
order_picker_conf.display_and_describe_configs()


Current Configuration Settings for OrderPicker:
{'start_date': datetime.date(2020, 1, 1), 'end_date': datetime.date(2024, 1, 1)}
        

Configuration Descriptions for OrderPicker:
- start_date: 2020-01-01  # The start date for selecting orders.
- end_date: 2024-01-01  # The end date for selecting orders.



In [14]:
@pydantic_dataclass
class BaseSizerConfigs(BaseConfigs, ABC):
    """
    Base configuration class for Sizer modules.
    """
    pass

@pydantic_dataclass
class DefaultSizerConfigs(BaseSizerConfigs):
    """
    Default configuration class for Sizer modules.
    """
    sizing_lev: float = 1.0

@pydantic_dataclass
class ZscoreSizerConfigs(BaseSizerConfigs):
    """
    Z-score based configuration class for Sizer modules.
    """
    sizing_lev: float = 1.0
    rvol_window: int|tuple = None
    rolling_window: int = 100
    weights: Tuple[float, float, float] = (0.5, 0.3, 0.2)
    vol_type:str = 'mean'
    norm_const:float = 1.0

default_sizer_conf = ZscoreSizerConfigs()